In [1]:
import pandas as pd 
import numpy as np
import logging
import os
import sys
import seaborn as sns
import matplotlib.pyplot as plt



In [2]:
from Common_Utils import load_yaml,extract_zip,transfer_file
from Common_Utils.file_operations import FileReader

dict_file = load_yaml("F:\ML_project\config_path.yaml")
zip_path = dict_file["DataIngestion"]["zip_path"]
collected_data_path = dict_file["DataIngestion"]["collected_data_path"]
extracted_data_path = dict_file["DataIngestion"]["extracted_data_path"]
data_artifacts_path = dict_file["DataIngestion"]["data_artifacts_path"]


extract_zip(zip_path= zip_path, extract_to= collected_data_path)

transfer_file(source_path= extracted_data_path, destination_path= data_artifacts_path )



In [3]:
from Common_Utils.file_operations import FileReader

raw_data_path = dict_file["DataValidation"]["raw_data_path"]

df = FileReader.read_file(file_path= raw_data_path)
df['Churn'] = df['Attrition_Flag'].apply(lambda val: 0 if val == "Existing Customer" else 1)
df.drop(columns=df.columns[[0,1]], axis=1, inplace=True)
df.head()

,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Churn
0,45,M,3,High School,Married,$60K - $80K,Blue,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0
1,49,F,5,Graduate,Single,Less than $40K,Blue,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0
2,51,M,3,Graduate,Married,$80K - $120K,Blue,36,4,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0
3,40,F,4,High School,Unknown,Less than $40K,Blue,34,3,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0
4,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,5,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0


In [4]:
from Common_Utils.dataframe_methods import DataFrameMethods
num_col= DataFrameMethods.numerical_column_list(df)
cat_col = DataFrameMethods.categorical_column_list(df)
print(DataFrameMethods.missing_values_columns(df))
print(DataFrameMethods.unique_values(df,["object","category"]))
print(DataFrameMethods.column_category_percentage(df,"Churn"))

Empty DataFrame
Columns: [Column Name, Missing Value Percentage, Data Type]
Index: []
       column_name  no.                                             values  \
1  Education_Level    7  [High School, Graduate, Uneducated, Unknown, C...   
3  Income_Category    6  [$60K - $80K, Less than $40K, $80K - $120K, $4...   
2   Marital_Status    4               [Married, Single, Unknown, Divorced]   
4    Card_Category    4                     [Blue, Gold, Silver, Platinum]   
0           Gender    2                                             [M, F]   

             mode  
1        Graduate  
3  Less than $40K  
2         Married  
4            Blue  
0               F  
   category  Count          %
0         0   8500  83.934038
1         1   1627  16.065962


In [5]:
df_numcol_stats =DataFrameMethods.descriptive_statistics(df,"churn")
df_numcol_stats.round(2)

,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Churn
mean,46.33,2.35,35.93,3.81,2.34,2.46,8631.95,1162.81,7469.14,0.76,4404.09,64.86,0.71,0.27,0.16
median,46.00,2.00,36.00,4.00,2.00,2.00,4549.00,1276.00,3474.00,0.74,3899.00,67.00,0.70,0.18,0.00
variance,64.27,1.69,63.78,2.42,1.02,1.22,82605861.00,664204.36,82640559.65,0.05,11540487.17,550.96,0.06,0.08,0.13
std_dev,8.02,1.30,7.99,1.55,1.01,1.11,9088.78,814.99,9090.69,0.22,3397.13,23.47,0.24,0.28,0.37
skewness,-0.03,-0.02,-0.11,-0.16,0.63,0.01,1.67,-0.15,1.66,1.73,2.04,0.15,2.06,0.72,1.85
kurtosis,-0.29,-0.68,0.40,-1.01,1.10,0.00,1.81,-1.15,1.80,9.99,3.89,-0.37,15.69,-0.79,1.42
min,26.00,0.00,13.00,1.00,0.00,0.00,1438.30,0.00,3.00,0.00,510.00,10.00,0.00,0.00,0.00
25%,41.00,1.00,31.00,3.00,2.00,2.00,2555.00,359.00,1324.50,0.63,2155.50,45.00,0.58,0.02,0.00
50%,46.00,2.00,36.00,4.00,2.00,2.00,4549.00,1276.00,3474.00,0.74,3899.00,67.00,0.70,0.18,0.00
75%,52.00,3.00,40.00,5.00,3.00,3.00,11067.50,1784.00,9859.00,0.86,4741.00,81.00,0.82,0.50,0.00


In [6]:
from Model_Utils.Data_Preprocessing.feature_outlier_detection import FeatureOutlierDetection
df_outlier_detected, outlier_nums_col  = FeatureOutlierDetection.detect_outliers_in_df(df,'zscore',"Churn")
print(df_outlier_detected)
print(outlier_nums_col)

   index             column_name  number_of_outliers  percentage_of_outliers  \
0     15         Total_Trans_Amt                 391                3.860966   
1     14    Total_Amt_Chng_Q4_Q1                 163                1.609559   
2      9  Months_Inactive_12_mon                 124                1.224449   
3     17     Total_Ct_Chng_Q4_Q1                 113                1.115829   
4     10   Contacts_Count_12_mon                  54                0.533228   
5     16          Total_Trans_Ct                   2                0.019749   
6      6           Card_Category                   2                0.019749   
7      0            Customer_Age                   1                0.009875   
8      3         Education_Level                   1                0.009875   

  column_dtype  
0        int64  
1      float64  
2        int64  
3      float64  
4        int64  
5        int64  
6       object  
7        int64  
8       object  
['Customer_Age' 'Dependent_co

In [7]:
from Model_Utils.Data_Preprocessing.feature_outlier_handling import FeatureOutlierTreatment
df_outlier_handled = FeatureOutlierTreatment.handling_outliers_df(df,outlier_nums_col,'yeo_johnson')
print(df_outlier_handled)

       Customer_Age Gender  Dependent_count Education_Level Marital_Status  \
0         49.749338      M         2.743626     High School        Married   
1         54.317336      F         4.430543        Graduate         Single   
2         56.606106      M         2.743626        Graduate        Married   
3         44.058638      F         3.596112     High School        Unknown   
4         44.058638      M         2.743626      Uneducated        Married   
...             ...    ...              ...             ...            ...   
10122     55.461336      M         1.867926        Graduate         Single   
10123     45.194962      M         1.867926         Unknown       Divorced   
10124     48.609422      F         0.959991     High School        Married   
10125     32.751886      M         1.867926        Graduate        Unknown   
10126     47.470374      F         1.867926        Graduate        Married   

      Income_Category Card_Category  Months_on_book  Total_Rela

In [8]:
from Model_Utils.Data_Preprocessing.feature_encoding import FeatureEncodingMethods

ordinal_cols = ['Card_Category', 'Education_Level', 'Income_Category']
categories = {"Education_Level":['Unknown', 'Uneducated', 'High School', 'College', 'Graduate', 'Post-Graduate', 'Doctorate'],
              "Income_Category":['Unknown', 'Less than $40K', '$40K - $60K', '$60K - $80K', '$80K - $120K', '$120K +'],
              "Card_Category":['Blue', 'Silver', 'Gold', 'Platinum'],
              }
ohe_cols = ['Gender', 'Marital_Status']
df_feature_encoded = FeatureEncodingMethods.feature_encoded_df(df_outlier_handled,ohe_cols,'onehot') 
df_feature_encoded = FeatureEncodingMethods.feature_encoded_df(df_feature_encoded,ordinal_cols,'ordinal', categories) 

print(df_feature_encoded.head())
print(DataFrameMethods.numerical_column_list(df_feature_encoded))

encoded_columns = [col for col in df_feature_encoded.columns if col not in df.columns]
removed_columns_after_encoding = [col for col in df.columns if col not in df_feature_encoded.columns]

print(f'"new encoded {len(encoded_columns)} columns added : {encoded_columns}"')
print(f'" removed {len(removed_columns_after_encoding)} columns from original dataset : {removed_columns_after_encoding}"')

   Customer_Age  Dependent_count  Education_Level  Income_Category  \
0     49.749338         2.743626              2.0              3.0   
1     54.317336         4.430543              4.0              1.0   
2     56.606106         2.743626              4.0              4.0   
3     44.058638         3.596112              2.0              1.0   
4     44.058638         2.743626              1.0              3.0   

   Card_Category  Months_on_book  Total_Relationship_Count  \
0            0.0       61.721466                  5.286987   
1            0.0       70.900926                  6.381547   
2            0.0       56.301071                  4.201241   
3            0.0       52.726517                  3.126001   
4            0.0       30.381384                  5.286987   

   Months_Inactive_12_mon  Contacts_Count_12_mon  Credit_Limit  ...  \
0                0.778559               2.916317      3.246818  ...   
1                0.778559               1.957234      3.216614  

In [9]:
X_train, X_test, y_train, y_test,X,y = DataFrameMethods.split_train_test(df= df_feature_encoded, target_column= "Churn", test_size= 0.3, random_state= 42)

In [13]:
from Model_Utils.Data_Preprocessing.feature_sampling import FeatureOverSamplingMethods
X_train_resampled, y_train_resampled, df_train_resampled = FeatureOverSamplingMethods.oversampled_balanced_df(X_train, y_train, X, "Churn", "smote")

In [14]:
from Model_Utils.Data_Preprocessing.feature_scaling import FeatureScalingMethods
X_train_scaled, X_test_scaled, df_train_scaled, df_test_scaled = FeatureScalingMethods.feature_scaled_df(X_train_resampled, X_test, y_train_resampled, y_test, X,"standardization")

In [25]:
print(X_train_scaled.shape)
print(y_train_resampled.shape)
X_train1 = pd.read_csv("F:\ML_project\Data\data_artifacts\X_train.csv")
y_train1 = pd.read_csv("F:\ML_project\Data\data_artifacts\y_train.csv").squeeze("columns")
print(X_train1.shape)
print(y_train1.shape)





(11898, 21)
(11898,)
(11898, 21)
(11898,)


In [12]:
from Model_Utils.Model_Selection.supervised_tuned_models import SupervisedHyperparameterSearchMethods
from Model_Utils.Model_Selection.models_initialization import SelectedModelInitializer
dict_sup_tuned_hyperparams = SupervisedHyperparameterSearchMethods.tuned_model_parameters(models=SelectedModelInitializer.classifier,
                                                                                        X_train= X_train_scaled , y_train=y_train_resampled,
                                                                                        scoring= "accuracy",
                                                                                        yaml_file=dict_file["ModelTrainingEvaluation"]["classifiers_param_path"],
                                                                                        chosen_strategy="grid_search_cv")

In [13]:
from Model_Utils.Model_Selection.Models_training_evaluation import BestModelEvaluation
df_tuned_models= BestModelEvaluation.evaluate(model_dict=SelectedModelInitializer.classifier,
                                    param_dict=dict_sup_tuned_hyperparams,
                                    X_train= X_train_scaled,
                                    X_test= X_test_scaled,
                                    y_train= y_train_resampled,
                                    y_test= y_test,
                                    scoring= "accuracy_test"
                                    )
print(df_tuned_models)



                    Model  accuracy_train  accuracy_test  precision_train  \
0      LogisticRegression        0.884182       0.868378         0.884238   
1  DecisionTreeClassifier        1.000000       0.923988         1.000000   

   precision_test  recall_train  recall_test  f1_Score_train  f1_Score_test  \
0        0.897890      0.884182     0.868378        0.884178       0.877640   
1        0.925855      1.000000     0.923988        1.000000       0.924809   

   roc_auc_train  roc_auc_test  
0       0.953082      0.932277  
1       1.000000      0.870208  


In [14]:
from Model_Utils.Model_Selection.Models_training_evaluation import SelectedModelsYaml
SelectedModelsYaml.tuned_models_yaml(models_dict=SelectedModelInitializer.classifier,
                                     params_dict=dict_sup_tuned_hyperparams,
                                     metrics_df=df_tuned_models,
                                     yaml_path=dict_file["ModelTrainingEvaluation"]["tuned_models_path"],
                                     scoring="accuracy_test")
                                     
                                     
                                     

In [15]:
type(y_train)

pandas.core.series.Series

In [7]:
import joblib
import pathlib
from pathlib import Path

# Load the saved model
model = joblib.load(Path("F:/ML_project/Final_Model/2025-04-07_17-07-05/model.joblib"))  # Update path

# Check if the model has get_params (works for most sklearn models/pipelines)
if hasattr(model, "get_params"):
    params = model.get_params()
    filtered_params = {k: v for k, v in params.items() if v is not None and v != 0}
    for k, v in filtered_params.items():
        print(f"{k}: {v}")  # Replace with MLflow logging or write to file
else:
    print("This model does not support get_params()")

criterion: entropy
max_depth: 20
min_samples_leaf: 1
min_samples_split: 5
splitter: best


c:\Users\prasa\anaconda3\envs\myenv\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [15]:
from Common_Utils import load_yaml
models_config = load_yaml("F:/ML_project/Yaml_Repo/tuned_params_metrics.yaml")

first_model_name = next(iter(models_config))
model_entry = models_config[first_model_name]
model_str = model_entry["model"]
model_params = model_entry["parameters"]
print(model_params)


{'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2}


In [1]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Set local tracking URI
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# Optional: set experiment name (will create if not exists)
mlflow.set_experiment("local_test_experiment")

# Load data
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)

# Log everything
with mlflow.start_run():
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("random_state", 42)
    mlflow.log_metric("accuracy", acc)
    mlflow.sklearn.log_model(model, "model")

    print("✅ Run logged to MLflow with accuracy:", acc)


2025/04/07 19:50:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Run logged to MLflow with accuracy: 1.0
🏃 View run righteous-carp-649 at: http://127.0.0.1:5000/#/experiments/638471175029152369/runs/14dba1288d504d7ca4aa5cb2a5bb043c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/638471175029152369


In [2]:
import sys
from Common_Utils import logger, CustomException


def divide_numbers(a, b):
    """Divides two numbers and logs errors using CustomException."""
    try:
        result = a / b
        logger.info(f"Result: {result}")
        return result
    except Exception as e:
        raise CustomException(e, sys)  # Raise Custom Exception with details

# Example Usage
try:
    divide_numbers(10, 10)  # Will cause division by zero
except CustomException as ce:
    logger.error(f"Exception found: {ce}")

ModuleNotFoundError: No module named 'Common_Utils'

In [ ]:
client.transition_model_version_stage(
                name=model_name,
                version=model_version,
                stage=new_stage,
                archive_existing_versions=True
            )

versions = client.get_latest_versions(model_name, stages=["Staging"])

In [2]:
import numpy as np 
import pandas as pd

df = pd.read_csv("F:\ML_project\Data\data_artifacts\X_test.csv")
df.dtypes


Customer_Age                float64
Dependent_count             float64
Education_Level             float64
Income_Category             float64
Card_Category               float64
Months_on_book              float64
Total_Relationship_Count    float64
Months_Inactive_12_mon      float64
Contacts_Count_12_mon       float64
Credit_Limit                float64
Total_Revolving_Bal         float64
Avg_Open_To_Buy             float64
Total_Amt_Chng_Q4_Q1        float64
Total_Trans_Amt             float64
Total_Trans_Ct              float64
Total_Ct_Chng_Q4_Q1         float64
Avg_Utilization_Ratio       float64
Gender_M                    float64
Marital_Status_Married      float64
Marital_Status_Single       float64
Marital_Status_Unknown      float64
dtype: object

In [2]:
input_json = {
    "Customer_Age": 45,
    "Gender": "M",
    "Dependent_count": 2,
    "Education_Level": "Graduate",
    "Marital_Status": "Married",
    "Income_Category": "$40K - $60K",
    "Card_Category": "Blue",
    "Months_on_book": 36,
    "Total_Relationship_Count": 3,
    "Months_Inactive_12_mon": 1,
    "Contacts_Count_12_mon": 3,
    "Credit_Limit": 7500.0,
    "Total_Revolving_Bal": 500,
    "Avg_Open_To_Buy": 7000.0,
    "Total_Amt_Chng_Q4_Q1": 1.2,
    "Total_Trans_Amt": 10000,
    "Total_Trans_Ct": 75,
    "Total_Ct_Chng_Q4_Q1": 1.0,
    "Avg_Utilization_Ratio": 0.3
  }

In [6]:
import pandas as pd
import joblib

# Load encoder
encoder = joblib.load(r"F:\ML_project\Script\ordinal_encoder.joblib")

# Sample input with both categorical (ordinal) and numerical columns
input_dict = {
    "Customer_Age": 45,
    "Dependent_count": 3,
    "Education_Level": "Graduate",
    "Income_Category": "$60K - $80K",
    "Card_Category": "Gold",
    "Months_on_book": 39,
    "Total_Relationship_Count": 5,
    "Months_Inactive_12_mon": 1,
    "Contacts_Count_12_mon": 3,
    "Credit_Limit": 12000.0,
    "Total_Revolving_Bal": 1200,
    "Avg_Open_To_Buy": 10800.0,
    "Total_Amt_Chng_Q4_Q1": 1.3,
    "Total_Trans_Amt": 4500,
    "Total_Trans_Ct": 90,
    "Total_Ct_Chng_Q4_Q1": 1.0,
    "Avg_Utilization_Ratio": 0.1
}

# Convert to DataFrame
input_df = pd.DataFrame([input_dict])

# Specify ordinal columns and order
ordinal_cols = ['Card_Category', 'Education_Level', 'Income_Category']
non_ordinal_df = input_df.drop(columns=ordinal_cols)

# Transform ordinal columns
encoded_array = encoder.transform(input_df[ordinal_cols])
encoded_df = pd.DataFrame(encoded_array, columns=ordinal_cols)

# Concatenate encoded + numeric columns
final_df = pd.concat([non_ordinal_df.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)

print(final_df)


   Customer_Age  Dependent_count  Months_on_book  Total_Relationship_Count  \
0            45                3              39                         5   

   Months_Inactive_12_mon  Contacts_Count_12_mon  Credit_Limit  \
0                       1                      3       12000.0   

   Total_Revolving_Bal  Avg_Open_To_Buy  Total_Amt_Chng_Q4_Q1  \
0                 1200          10800.0                   1.3   

   Total_Trans_Amt  Total_Trans_Ct  Total_Ct_Chng_Q4_Q1  \
0             4500              90                  1.0   

   Avg_Utilization_Ratio  Card_Category  Education_Level  Income_Category  
0                    0.1            2.0              4.0              3.0  


c:\Users\prasa\anaconda3\envs\myenv\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator OrdinalEncoder from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
import joblib
import pandas as pd

# Load saved encoder
encoder = joblib.load("F:\ML_project\Script\ohe_encoder.joblib")

# Sample input — single row (same structure as training)
input_dict = {
    "Gender": ["Male"],
    "Marital_Status": ["Single"],
    "Customer_Age": [45],
    "Dependent_count": [3],
    "Education_Level": ["Graduate"],
    "Income_Category": ["$60K - $80K"],
    "Card_Category": ["Blue"],
    "Months_on_book": [39],
    "Total_Relationship_Count": [5],
    "Months_Inactive_12_mon": [1],
    "Contacts_Count_12_mon": [3],
    "Credit_Limit": [3000.0],
    "Total_Revolving_Bal": [1500],
    "Avg_Open_To_Buy": [1500.0],
    "Total_Amt_Chng_Q4_Q1": [0.8],
    "Total_Trans_Amt": [5000],
    "Total_Trans_Ct": [70],
    "Total_Ct_Chng_Q4_Q1": [0.7],
    "Avg_Utilization_Ratio": [0.5]
}

input_df = pd.DataFrame(input_dict)

# Select categorical columns only to transform
categorical_cols = ['Gender', 'Marital_Status']
numerical_cols = [col for col in input_df.columns if col not in categorical_cols]

# Encode categorical features
encoded_cats = encoder.transform(input_df[categorical_cols])

# Get encoded feature names
encoded_cols = encoder.get_feature_names_out(categorical_cols)

# Build the full processed input
encoded_df = pd.DataFrame(encoded_cats, columns=encoded_cols)
numerical_df = input_df[numerical_cols].reset_index(drop=True)

final_input = pd.concat([numerical_df, encoded_df], axis=1)

print(final_input)

   Customer_Age  Dependent_count Education_Level Income_Category  \
0            45                3        Graduate     $60K - $80K   

  Card_Category  Months_on_book  Total_Relationship_Count  \
0          Blue              39                         5   

   Months_Inactive_12_mon  Contacts_Count_12_mon  Credit_Limit  ...  \
0                       1                      3        3000.0  ...   

   Avg_Open_To_Buy  Total_Amt_Chng_Q4_Q1  Total_Trans_Amt  Total_Trans_Ct  \
0           1500.0                   0.8             5000              70   

   Total_Ct_Chng_Q4_Q1  Avg_Utilization_Ratio  Gender_M  \
0                  0.7                    0.5       0.0   

   Marital_Status_Married  Marital_Status_Single  Marital_Status_Unknown  
0                     0.0                    1.0                     0.0  

[1 rows x 21 columns]


c:\Users\prasa\anaconda3\envs\myenv\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\prasa\anaconda3\envs\myenv\lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [10]:
input_df.dtypes

Customer_Age                float64
Dependent_count             float64
Education_Level             float64
Income_Category             float64
Card_Category               float64
Months_on_book              float64
Total_Relationship_Count    float64
Months_Inactive_12_mon      float64
Contacts_Count_12_mon       float64
Credit_Limit                float64
Total_Revolving_Bal         float64
Avg_Open_To_Buy             float64
Total_Amt_Chng_Q4_Q1        float64
Total_Trans_Amt             float64
Total_Trans_Ct              float64
Total_Ct_Chng_Q4_Q1         float64
Avg_Utilization_Ratio       float64
dtype: object

In [ ]:
input_df = FeatureEncodingMethods.feature_encoded_df(input_df,ordinal_cols,'ordinal', categories)
scaler = StandardScaler()
scaled_values = scaler.fit_transform(input_df)
input_df = pd.DataFrame(scaled_values, columns=input_df.columns)